## 네이버 뉴스 페이지에 있는 여러 탭을 크롤링하기
### 탭마다 구조가 다르므로 함수로 처리함

In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}

In [2]:
# url을 request하고 html 코드를 받아서 parsing하는 함수
def create_soup(url):
    res = req.get(url, headers = headers)
    soup = bs(res.text, 'html.parser')
    return soup

### 서로 다른 기능을 가진 탭별 크롤링 함수 생성 

### 1. 헤드라인 뉴스 : scrape_headline_news()
'정치', '경제', '사회', '생활/문화', 'IT/과학', '세계'는 구조가 동일함

In [3]:
# '정치', '경제', '사회', '생활/문화', 'IT/과학', '세계'는 구조가 동일함
def scrape_headline_news(sid):
    url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=' + str(sid)
    soup = create_soup(url)
    
    atags = soup.select('ul.sh_list > li > div.sh_text > a')
    contents = soup.select('ul.sh_list > li > div.sh_text > div.sh_text_lede')

    for i in range(len(atags)):        
        title = atags[i].text
        link = atags[i].get('href')
        content = contents[i].text
        print('{}. {}'.format(i+1, title))
        print(link)
        print(content)
        print()

### 2. 랭킹 : scrape_ranking()

In [4]:
def scrape_ranking(papername):
    
    url = 'https://news.naver.com/main/ranking/popularDay.naver?mid=etc&sid1=111'
    soup = create_soup(url)
    
    soup_all = soup.find_all(class_ = re.compile('^_officeCard _officeCard'))
    break_flg = False
    for page in soup_all:
        newspapers = page.select('div.rankingnews_box')
        for newspaper in newspapers:
            atags = newspaper.select('a > strong')
            for atag in atags:
                if atag.text == papername:
                    print(atag.text)
                    get_ranking_news(newspaper)
                    
                    break_flg = True  # 검색대상 언론사를 찾으면 작업 후 빠져 나간다.
                    break
            if break_flg:
                break
        if break_flg:
            break

def get_ranking_news(newspaper):
    articles = newspaper.select("a.list_title")

    for i in range(len(articles)):
        title = articles[i].text
        link = articles[i].get('href')
        print('{}. {}'.format(i+1, title))
        print(link)
        print()

### 2. 팩트체크 : scrape_fact_check()

In [27]:
def scrape_fact_check(max_n):
    
    url = 'https://news.naver.com/main/factcheck/main.naver'
    soup = create_soup(url)
    
    soup_section = soup.select('div.fact_lst._factcheck_body > div.section')
    for i in range(len(soup_section)):
        #print(soup_section[i])
        fact_list = soup_section[i].select_one('p.txt > a')
        print('-----'*15)
        print('체크 항목:', fact_list.text)
        print('-----'*15)
        
        result = soup_section[i].select_one('div.news.hide')
        print(result.text)
        print()
        
        if i >= max_n - 1:
            break
        

In [28]:
while True:
    #sid = input('검색하고자 하는 뉴스의 분야를 숫자로 입력하세요:\n언론사별:0 정치:1 경제:2 사회:3 생활/문화:4 세계:5 IT/과학:6 랭킹:7 신문보기:8 오피니언:9 팩트체크:10 종료:999\n')
    sid = '10'
    if not sid.isdigit():
        continue
    else:
        sid = int(sid)
        
    if sid == 999:
        break
    elif 0 < sid and sid < 7:
        scrape_headline_news(sid + 99)  # 정치: 100, 경제: 101 ...
    elif sid == 7:
        newspaper = input('언론사명을 입력하세요(예, 한국일보):')
        scrape_ranking(newspaper)
    elif sid == 10:
        scrape_fact_check(5)  # 최대 5개
    else:
        print('7부터 10까지')
    
    break

---------------------------------------------------------------------------
체크 항목: 수입 가구를 통해 집을 무너뜨리는 흰개미가 국내에 유입됐다
---------------------------------------------------------------------------


판단유보



[요약]마른 나무를 먹는 흰개미가 수입 가구를 통해 수년 전 국내로 유입된 뒤 이미 토착화된 게 아니냐는 우려가 커지고 있다. 발견된 외래 흰개미는 가구가 아닌 나무로 된 문틀에서 서식하던 것으로 추정된다.나무로 된 가구나 화분 속 흙 등을 통해 최소 5년 전 국내로 들어온 뒤 번식했을 가능성이 높다. 국내 기후는 마른 나무를 먹는 흰개미가 살기 어렵지만 실내나 땅 속에선 서식이 가능하다.외래 흰개미의 유입 경로를 특정하긴 쉽지 않고 추가 발견 가능성이 있어 판단을 유보한다. 

[검증대상]소셜네트워크서비스 등 온라인에서는 수입 가구를 통해 마른 나무를 먹는 흰개미가 서울 강남구 일대에 퍼졌다는 의혹이 제기됐다.▲온라인에서는 외래 흰개미가 수입 가구를 통해 들어왔다는 주장이 나왔다. 트위터 캡쳐 [검증방법]-환경부·전문가 인터뷰-흰개미 관련 자료[검증내용]환경부와 국립생태원은 지난 18일 외래 흰개미가 발견된 주택에서 사체 2개를 추가로 발견한 뒤 현미경으로 관찰한 결과 마른나무를 좋아하는 ‘크립토털미스’속 외래 흰개미로 추정된다고 밝혔다. 구체적인 종을 확인하기 위해선 유전자 분석이 필요한 상황이다. 집도 무너뜨리는 외래 흰개미?…국내는 내부 목재 자재나 가구 등 피해 우려 국립산림과학원의 '알기 쉬운 흰개미 방제 가이드북' 등에 따르면, 일본흰개미나 칸몬흰개미처럼 국내에 서식하는 흰개미는 수분이 있는 목재를 갉아먹었다. 이 때문에 습하고 그늘진 곳에 있는 목재 건축물들의 바닥 부분이 주로 피해를 입기 쉬웠다. 국내에선 나무의 수분 함량이 30%가 넘어가면, 레이저로 움직임을 탐지하는 식으로 흰개미를 추